# Import

In [1]:
import pandas as pd
import numpy as np
from importlib import reload
from astropy.io import fits
from forecast_SN_GW import get_hubblefit

# Load data

In [2]:
d_ps1 = pd.read_csv('../Data/SN_ps_snls_sdss_hst.csv', sep=' ', index_col='CID')
ps1_pd = d_ps1[d_ps1['IDSURVEY'] == 15]

d_X = pd.read_csv('../Data/jla_lcparams.txt', sep=' ', index_col='#name')

bias_data = fits.getdata('../Data/covmat/C_bias.fits')
cal_data = fits.getdata('../Data/covmat/C_cal.fits')
dust_data = fits.getdata('../Data/covmat/C_dust.fits')
host_data = fits.getdata('../Data/covmat/C_host.fits')
model_data = fits.getdata('../Data/covmat/C_model.fits')
nonia_data = fits.getdata('../Data/covmat/C_nonia.fits')
pecvel_data = fits.getdata('../Data/covmat/C_pecvel.fits')
stat_data = fits.getdata('../Data/covmat/C_stat.fits')
covX = bias_data + cal_data + dust_data + host_data + model_data + nonia_data + pecvel_data + stat_data

# Reproduce JLA results

## Build data array for JLA fit

In [3]:
mass = np.zeros_like(d_X['3rdvar'].values)
for i, m in enumerate(d_X['3rdvar'].values):
    if m > 10:
        mass[i] = -1.
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, mass])

d_sigma = pd.read_csv('../Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## JLA fit

In [4]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])

In [5]:
g.fit()

RuntimeError: Cannot understand keyword Pedantic. May be a typo?
The parameters are ['Mb', 'alpha1', 'alpha2', 'alpha3', 'omgM']

In [19]:
g.minuit.values

<ValueView of Minuit at 558dbd2ab328>
  Mb: -19.048500801549057
  alpha1: 0.1412273345463002
  alpha2: -3.101541148875581
  alpha3: -0.06995742381361028
  omgM: 0.295371105743009

In [20]:
g.minuit.errors

<ErrorView of Minuit at 558dbd2ab328>
  Mb: 0.023170305107844093
  alpha1: 0.006590085152848236
  alpha2: 0.08072799314194437
  alpha3: 0.023110783961262468
  omgM: 0.03393357277465823

# Fit with linear correction in redshift 

## Build data array 

In [7]:
X = np.array([d_X['mb'].values, d_X['x1'].values, d_X['color'].values, d_X['zcmb'].values])

d_sigma = pd.read_csv('../Data/covmat/sigma_mu.txt', sep=' ')
S = np.array([d_sigma['#sigma_coh'].values, d_sigma['#sigma_lens'].values,
              d_sigma['#z'].values])
S = S.T

## Fit 

In [8]:
g = get_hubblefit(X.T, covX, d_X['zcmb'].values, d_X['zcmb'].values, S[:, 0], S[:, 1])
g.fit()

/home/nora/Documents/Thèse/Missions/2019-08-18-Euclid/Cours/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter Mb is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/nora/Documents/Thèse/Missions/2019-08-18-Euclid/Cours/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha1 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/nora/Documents/Thèse/Missions/2019-08-18-Euclid/Cours/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha2 is floating but does not have initial step size. Assume 1.
  self.minuit = minuit.Minuit(self._minuit_chi2_, **minuit_kwargs)
/home/nora/Documents/Thèse/Missions/2019-08-18-Euclid/Cours/forecast_SN_GW/forecast_SN_GW/Hubble_fit.py:214: InitialParamWarning: Parameter alpha3 is floating but does not have initial 

(0.15357536939936012, 0.0050001990226806676)